<a href="https://colab.research.google.com/github/Pravallika1234uppala/ML/blob/main/ML%20Models/Supervised%20Learning/Classification%20%26%20Regression/Ensemble%20Techniques/Boosting/Xgboost/Xgboost%20Regression./Xgboost_Regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **XGboost REGRESSION:**

## Used Cars Price Prediction

1. Problem Statement:

In [1]:
#where we find the price of cars.
#these prices are suggested



2. Data Collection:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

%matplotlib inline


In [3]:
df = pd.read_csv('usedcars.csv', index_col=[0])

In [4]:
df

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19537,Hyundai i10,Hyundai,i10,9,10723,Dealer,Petrol,Manual,19.81,1086,68.05,5,250000
19540,Maruti Ertiga,Maruti,Ertiga,2,18000,Dealer,Petrol,Manual,17.50,1373,91.10,7,925000
19541,Skoda Rapid,Skoda,Rapid,6,67000,Dealer,Diesel,Manual,21.14,1498,103.52,5,425000
19542,Mahindra XUV500,Mahindra,XUV500,5,3800000,Dealer,Diesel,Manual,16.00,2179,140.00,7,1225000


**FEATURE ENGINEERING:**

3. Data Cleaning:

. Handling Missing values

. Handling Duplicates

. Check data type

. Understanding dataset

**. Handling Missing values:**

In [5]:
#checking null values:
#checking features with nan values

df.isnull().sum()

,0
car_name,0
brand,0
model,0
vehicle_age,0
km_driven,0
seller_type,0
fuel_type,0
transmission_type,0
mileage,0
engine,0


In [6]:
#removing unnessary columns:

#model is enough to find brand
df.drop(['car_name','brand'], axis=1, inplace=True)

In [7]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [8]:
df['model'].unique(), len(df['model'].unique())

(array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
        'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
        'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
        'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
        'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
        'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
        'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
        'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
        'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
        'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
        'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
        'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
        'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
        'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Gl

In [9]:
#cant add 120 columns as one hot encoding - better to do label encoding

#getting all types of features:

num_features = [feature for feature in df.columns if df[feature].dtype!='O']
print("Number of numerical features:", len(num_features))

cat_features = [feature for feature in df.columns if df[feature].dtype=='O']
print("Number of categorical features:", len(cat_features))

discrete_features = [feature for feature in num_features if len(df[feature].unique())<=25]
print("Number of discrete features:", len(discrete_features))

continuous_features = [feature for feature in num_features if feature not in discrete_features]
print("Number of continuous features:", len(continuous_features))



Number of numerical features: 7
Number of categorical features: 4
Number of discrete features: 2
Number of continuous features: 5


In [10]:
#dividing into input and output featiures:

X = df.drop(['selling_price'], axis=1)

y = df['selling_price']

In [11]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [12]:
y.head()

,selling_price
0,120000
1,550000
2,215000
3,226000
4,570000


### Feature Encoding & Scaling:

**. Check data type**

In [13]:
df.dtypes

,0
model,object
vehicle_age,int64
km_driven,int64
seller_type,object
fuel_type,object
transmission_type,object
mileage,float64
engine,int64
max_power,float64
seats,int64


In [14]:
df['model'].unique(), len(df['model'].unique())

(array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
        'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
        'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
        'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
        'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
        'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
        'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
        'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
        'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
        'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
        'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
        'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
        'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
        'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Gl

In [15]:
from sklearn.preprocessing import LabelEncoder

l = LabelEncoder()
X['model'] = l.fit_transform(X['model'])


In [16]:
df['seller_type'].unique()

array(['Individual', 'Dealer', 'Trustmark Dealer'], dtype=object)

In [17]:
df['fuel_type'].unique()

array(['Petrol', 'Diesel', 'CNG', 'LPG', 'Electric'], dtype=object)

In [18]:
df['transmission_type'].unique()

array(['Manual', 'Automatic'], dtype=object)

In [19]:
#3 types of transformers:

num_features = X.select_dtypes(exclude='object').columns
onehot_columns = ['seller_type', 'fuel_type', 'transmission_type']
#as there are only few categories

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
onehot_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
        ('One Hot Encoder', onehot_transformer, onehot_columns),
        ('Numerical Scaling', numeric_transformer, num_features)
    ], remainder='passthrough'
)


In [20]:
X = preprocessor.fit_transform(X)


In [21]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


**Train Test Split**

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)

X_train.shape, X_test.shape

((11558, 14), (3853, 14))

In [23]:
!pip install xgboost

**MODEL SELECTION:**

In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

#function to evaluate model:
def evaluate_model(true_value, predicted_value):
  mae = mean_absolute_error(true_value, predicted_value)
  mse = mean_squared_error(true_value, predicted_value)
  rmse = np.sqrt(mean_squared_error(true_value, predicted_value))
  r2_squa = r2_score(true_value, predicted_value)
  return mae, rmse, r2_squa

#key value pairs of model name and models
models = {
    "Linear Regression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "k Nearest Neighbour":KNeighborsRegressor(),
    "Decision Tree":DecisionTreeRegressor(),
    "Random Forest":RandomForestRegressor(),
    "Adaboost Regressor":AdaBoostRegressor(),
    "Gradient Boost Regressor":GradientBoostingRegressor(),
    "XG Boost Regressor":XGBRegressor()
    #this is based on default parameters
}

for i in range(len(list(models))):
  #taking particular model based on iteration
  model = list(models.values())[i]

  #training model
  model.fit(X_train, y_train)

  #making predictions:
  y_train_pred = model.predict(X_train)
  y_test_pred = model.predict(X_test)

  #evaluation: for training
  model_train_mae, model_train_rmse, model_train_r2squ = evaluate_model(y_train, y_train_pred)


  #evaluation: for testing
  model_test_mae, model_test_rmse, model_test_r2squ = evaluate_model(y_test, y_test_pred)

  print(list(models.keys())[i])

  print("Model performance for Training data:")
  print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
  print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
  print("- R2-Score: {:.4f}".format(model_train_r2squ))

  print('-------------------------------------------------')

  print("Model performance for Testing data:")
  print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
  print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
  print("- R2-Score: {:.4f}".format(model_test_r2squ))

  print("="*50)
  print("\n")


Linear Regression
Model performance for Training data:
- Root Mean Squared Error: 552154.2495
- Mean Absolute Error: 266675.1076
- R2-Score: 0.6220
-------------------------------------------------
Model performance for Testing data:
- Root Mean Squared Error: 519891.2635
- Mean Absolute Error: 284283.4460
- R2-Score: 0.6525


Lasso
Model performance for Training data:
- Root Mean Squared Error: 552154.2607
- Mean Absolute Error: 266674.0472
- R2-Score: 0.6220
-------------------------------------------------
Model performance for Testing data:
- Root Mean Squared Error: 519890.5730
- Mean Absolute Error: 284283.7890
- R2-Score: 0.6525


Ridge
Model performance for Training data:
- Root Mean Squared Error: 552154.8775
- Mean Absolute Error: 266635.3662
- R2-Score: 0.6220
-------------------------------------------------
Model performance for Testing data:
- Root Mean Squared Error: 519880.3842
- Mean Absolute Error: 284241.1129
- R2-Score: 0.6525


k Nearest Neighbour
Model performance

Hyperparameter tuning:

In [25]:
rfparams = {"max_depth":[5,None],
            "max_features":[5,15,"auto"],
            "min_samples_split":[5,20],
            "n_estimators":[100,250]}
xgboost_params = {
    "learning_rate": [0.1, 0.01],
    "n_estimators": [100,200,300],
    "max_depth": [5,8,12,20,30],
    "colsample_bytree": [0.5,0.8,1,0.3,0.4]
}

In [26]:
xgboost_params

{'learning_rate': [0.1, 0.01],
 'n_estimators': [100, 200, 300],
 'max_depth': [5, 8, 12, 20, 30],
 'colsample_bytree': [0.5, 0.8, 1, 0.3, 0.4]}

In [27]:
rfparams

{'max_depth': [5, None],
 'max_features': [5, 15, 'auto'],
 'min_samples_split': [5, 20],
 'n_estimators': [100, 250]}

In [28]:
#models for hyperparameter tuning:

randomcvmodels = [
('Random Forest Regressor', RandomForestRegressor(), rfparams),
                  ("XGboost Regressor", XGBRegressor(), xgboost_params)
                  ]


In [29]:
randomcvmodels

[('Random Forest Regressor',
  RandomForestRegressor(),
  {'max_depth': [5, None],
   'max_features': [5, 15, 'auto'],
   'min_samples_split': [5, 20],
   'n_estimators': [100, 250]}),
 ('XGboost Regressor',
  XGBRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, random_state=None, ...),
  {'learning_rate': [0

In [30]:
from sklearn.model_selection import RandomizedSearchCV

#stores as key value pair
model_params = {}
for name, model, params in randomcvmodels:
  random_model = RandomizedSearchCV(estimator=model, param_distributions=params, n_iter=100, cv=3, verbose=2, n_jobs=-1)
  random_model.fit(X_train, y_train)
  #for that key value is assigned
  model_params[name]=random_model.best_params_

for model_name in model_params:
  print(f"------------- Best Params for {model_name} ------")
  print(model_params[model_name])

Fitting 3 folds for each of 24 candidates, totalling 72 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
------------- Best Params for Random Forest Regressor ------
{'n_estimators': 100, 'min_samples_split': 5, 'max_features': 5, 'max_depth': None}
------------- Best Params for XGboost Regressor ------
{'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1, 'colsample_bytree': 0.8}


In [31]:
#key value pairs of model name and models
models = {

    "Random Forest":RandomForestRegressor(n_estimators=100, min_samples_split=5, max_features=5, max_depth=None),
    "Xgboost Regressor":XGBRegressor(learning_rate=0.1, n_estimators=200, max_depth=5, colsample_bytree=0.8)
    #this is based on best parameters
}

for i in range(len(list(models))):
  #taking particular model based on iteration
  model = list(models.values())[i]

  #training model
  model.fit(X_train, y_train)

  #making predictions:
  y_train_pred = model.predict(X_train)
  y_test_pred = model.predict(X_test)

  #evaluation: for training
  model_train_mae, model_train_rmse, model_train_r2squ = evaluate_model(y_train, y_train_pred)


  #evaluation: for testing
  model_test_mae, model_test_rmse, model_test_r2squ = evaluate_model(y_test, y_test_pred)

  print(list(models.keys())[i])

  print("Model performance for Training data:")
  print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
  print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
  print("- R2-Score: {:.4f}".format(model_train_r2squ))

  print('-------------------------------------------------')

  print("Model performance for Testing data:")
  print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
  print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
  print("- R2-Score: {:.4f}".format(model_test_r2squ))

  print("="*50)
  print("\n")

Random Forest
Model performance for Training data:
- Root Mean Squared Error: 184269.1533
- Mean Absolute Error: 58589.3625
- R2-Score: 0.9579
-------------------------------------------------
Model performance for Testing data:
- Root Mean Squared Error: 235406.0969
- Mean Absolute Error: 102161.2839
- R2-Score: 0.9287


Xgboost Regressor
Model performance for Training data:
- Root Mean Squared Error: 119626.5720
- Mean Absolute Error: 77686.9531
- R2-Score: 0.9823
-------------------------------------------------
Model performance for Testing data:
- Root Mean Squared Error: 337391.7325
- Mean Absolute Error: 106335.1250
- R2-Score: 0.8536


